In [1]:
import pandas as pd
df = pd.read_csv("movie.csv")
df

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [2]:
df.label.value_counts()

label
0    20019
1    19981
Name: count, dtype: int64

In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load("en_core_web_sm")
def preprocess(text):
    doc = nlp(text)
    
    no_stop_word = []
    for token in doc:
        if not token.is_stop and not token.is_punct:
            no_stop_word.append(token.lemma_)
    return " ".join(no_stop_word)
df["No_stop_word"] = df["text"].apply(preprocess)

In [4]:
 from sklearn.model_selection import train_test_split
x_train, x_test , y_train , y_test = train_test_split(df.No_stop_word, df.label , test_size= 0.2 , stratify= df.label )

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

In [6]:
movie_review = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("nb", MultinomialNB())
])
movie_review.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [7]:
y_pred = movie_review.predict(x_test)
   
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.87      0.85      4004
           1       0.87      0.82      0.84      3996

    accuracy                           0.85      8000
   macro avg       0.85      0.85      0.85      8000
weighted avg       0.85      0.85      0.85      8000



In [8]:
movie_review.score(x_test,y_test)

0.848

In [9]:
import pickle
with open("movie_review.pickle","wb") as f:
    pickle.dump(movie_review,f)